## Agent Executor

In [1]:
String userHomeDir = System.getProperty("user.home");
String localRespoUrl = "file://" + userHomeDir + "/.m2/repository/";
String langchain4jVersion = "0.35.0"

add local maven repository

In [ ]:
%dependency /add-repo local \{localRespoUrl} release|never snapshot|always
%dependency /list-repos


Remove installed package from Jupiter cache

In [3]:
%%bash 
rm -rf \{userHomeDir}/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/bsc/langgraph4j

Install required maven dependencies

In [ ]:
%dependency /add org.slf4j:slf4j-jdk14:2.0.9
%dependency /add org.bsc.langgraph4j:langgraph4j-core-jdk8:1.0-SNAPSHOT
%dependency /add org.bsc.langgraph4j:langgraph4j-langchain4j:1.0-SNAPSHOT
%dependency /add org.bsc.langgraph4j:langgraph4j-agent-executor:1.0-SNAPSHOT
%dependency /add dev.langchain4j:langchain4j:\{langchain4jVersion}
%dependency /add dev.langchain4j:langchain4j-open-ai:\{langchain4jVersion}

%dependency /resolve

Initialize Logger

In [5]:
try( var file = new java.io.FileInputStream("./logging.properties")) {
    var lm = java.util.logging.LogManager.getLogManager();
    lm.checkAccess(); 
    lm.readConfiguration( file );
}

var log = org.slf4j.LoggerFactory.getLogger("AgentExecutor");


In [6]:
import dev.langchain4j.agent.tool.P;
import dev.langchain4j.agent.tool.Tool;

import java.util.Optional;

import static java.lang.String.format;

public class TestTool {
    private String lastResult;

    Optional<String> lastResult() {
        return Optional.ofNullable(lastResult);
    }

    @Tool("tool for test AI agent executor")
    String execTest(@P("test message") String message) {

        lastResult = format( "test tool executed: %s", message);
        return lastResult;
    }
}


In [17]:
import static org.bsc.langgraph4j.StateGraph.END;
import static org.bsc.langgraph4j.StateGraph.START;
import org.bsc.langgraph4j.CompileConfig;
import org.bsc.langgraph4j.RunnableConfig;
import org.bsc.langgraph4j.checkpoint.BaseCheckpointSaver;
import org.bsc.langgraph4j.checkpoint.MemorySaver;
import org.bsc.langgraph4j.state.AgentState;
import org.bsc.langgraph4j.serializer.StateSerializer;
import org.bsc.langgraph4j.serializer.MapSerializer;

import org.bsc.langgraph4j.agentexecutor.AgentExecutor;

import dev.langchain4j.model.openai.OpenAiChatModel;

import java.util.List;
import java.util.Map;
import java.util.stream.Collectors;



var chatLanguageModel = OpenAiChatModel.builder()
    .apiKey( System.getenv("OPENAI_API_KEY") )
    //.modelName( "gpt-3.5-turbo-0125" )
    .modelName( "gpt-4o-mini" )
    .logResponses(true)
    .maxRetries(2)
    .temperature(0.0)
    .maxTokens(2000)
    .build();

var agentExecutor = new AgentExecutor();

var stateGraph = agentExecutor.graphBuilder()
        .chatLanguageModel(chatLanguageModel)
        .objectsWithTools(List.of(new TestTool()))
        .build();




### Test 1 
Update State replacing the 'input'

In [18]:
var saver = new MemorySaver();

CompileConfig compileConfig = CompileConfig.builder()
                .checkpointSaver( saver )
                .build();

var graph = stateGraph.compile( compileConfig );

var config = RunnableConfig.builder()
                .threadId("test1")
                .build();    
                
var iterator = graph.streamSnapshots( Map.of( "input", "perform test once" ), config );  

for( var step : iterator ) {
    log.info( "STEP: {}", step );
}



2024-10-05 00:18:05 FINEST org.bsc.langgraph4j.CompiledGraph$AsyncNodeGenerator <init> START


INTERMEDIATE STEPS: []


2024-10-05 00:18:06 INFO REPL.$JShell$59 do_it$ STEP: NodeOutput{node=__START__, state={input=perform test once, intermediate_steps=[]}}
2024-10-05 00:18:06 INFO REPL.$JShell$59 do_it$ STEP: StateSnapshot{node=agent, state={input=perform test once, intermediate_steps=[], agent_outcome=AgentOutcome[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_zd3mR7smO9qgLVgRH0mumttL", name = "execTest", arguments = "{"message":"perform test once"}" }, log=], finish=null]}, config=RunnableConfig(threadId=test1, checkPointId=ecd9df27-7a85-4431-a1b3-cb86239decca, nextNode=action)}


INTERMEDIATE STEPS: [IntermediateStep[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_zd3mR7smO9qgLVgRH0mumttL", name = "execTest", arguments = "{"message":"perform test once"}" }, log=], observation=test tool executed: perform test once]]


2024-10-05 00:18:07 INFO REPL.$JShell$59 do_it$ STEP: StateSnapshot{node=action, state={input=perform test once, intermediate_steps=[IntermediateStep[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_zd3mR7smO9qgLVgRH0mumttL", name = "execTest", arguments = "{"message":"perform test once"}" }, log=], observation=test tool executed: perform test once]], agent_outcome=AgentOutcome[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_zd3mR7smO9qgLVgRH0mumttL", name = "execTest", arguments = "{"message":"perform test once"}" }, log=], finish=null]}, config=RunnableConfig(threadId=test1, checkPointId=622c5973-813d-4809-bb27-d159c34e8cb7, nextNode=agent)}
2024-10-05 00:18:07 INFO REPL.$JShell$59 do_it$ STEP: StateSnapshot{node=agent, state={input=perform test once, intermediate_steps=[IntermediateStep[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_zd3mR7smO9qgLVgRH0mumttL", name = "execTest", arguments = "{"message":"per

In [19]:
var history = graph.getStateHistory(config).stream().collect( Collectors.toList() );

var state2 =  history.get(2);

var updatedConfig = graph.updateState( state2.config(), Map.of( "input", "perform test twice"), null);

var iterator = graph.streamSnapshots( null, updatedConfig );  

for( var step : iterator ) {
    log.info( "STEP:\n {}", step );
}


2024-10-05 00:18:07 FINEST org.bsc.langgraph4j.CompiledGraph$AsyncNodeGenerator <init> RESUME REQUEST
2024-10-05 00:18:07 FINEST org.bsc.langgraph4j.CompiledGraph$AsyncNodeGenerator <init> RESUME FROM agent


INTERMEDIATE STEPS: [IntermediateStep[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_zd3mR7smO9qgLVgRH0mumttL", name = "execTest", arguments = "{"message":"perform test once"}" }, log=], observation=test tool executed: perform test once]]


2024-10-05 00:18:08 INFO REPL.$JShell$60 do_it$ STEP:
 NodeOutput{node=action, state={input=perform test twice, intermediate_steps=[IntermediateStep[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_zd3mR7smO9qgLVgRH0mumttL", name = "execTest", arguments = "{"message":"perform test once"}" }, log=], observation=test tool executed: perform test once]], agent_outcome=AgentOutcome[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_zd3mR7smO9qgLVgRH0mumttL", name = "execTest", arguments = "{"message":"perform test once"}" }, log=], finish=null]}}
2024-10-05 00:18:08 INFO REPL.$JShell$60 do_it$ STEP:
 NodeOutput{node=agent, state={input=perform test twice, intermediate_steps=[IntermediateStep[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_zd3mR7smO9qgLVgRH0mumttL", name = "execTest", arguments = "{"message":"perform test once"}" }, log=], observation=test tool executed: perform test once]], agent_outcome=AgentOutcome[

INTERMEDIATE STEPS: [IntermediateStep[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_zd3mR7smO9qgLVgRH0mumttL", name = "execTest", arguments = "{"message":"perform test once"}" }, log=], observation=test tool executed: perform test once], IntermediateStep[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_0UIBwyGjHqlPWnVOV6tLunMX", name = "execTest", arguments = "{"message":"perform test again"}" }, log=], observation=test tool executed: perform test again]]


2024-10-05 00:18:09 INFO REPL.$JShell$60 do_it$ STEP:
 NodeOutput{node=action, state={input=perform test twice, intermediate_steps=[IntermediateStep[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_zd3mR7smO9qgLVgRH0mumttL", name = "execTest", arguments = "{"message":"perform test once"}" }, log=], observation=test tool executed: perform test once], IntermediateStep[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_0UIBwyGjHqlPWnVOV6tLunMX", name = "execTest", arguments = "{"message":"perform test again"}" }, log=], observation=test tool executed: perform test again]], agent_outcome=AgentOutcome[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_0UIBwyGjHqlPWnVOV6tLunMX", name = "execTest", arguments = "{"message":"perform test again"}" }, log=], finish=null]}}
2024-10-05 00:18:09 INFO REPL.$JShell$60 do_it$ STEP:
 NodeOutput{node=agent, state={input=perform test twice, intermediate_steps=[IntermediateStep[action=A

### Test 2
Update State replacing the 'input' using a cloned state

In [20]:

var config = RunnableConfig.builder()
                .threadId("test2")
                .build();    
                
var iterator = graph.streamSnapshots( Map.of( "input", "perform test once" ), config );  

for( var step : iterator ) {
    log.info( "STEP: {}", step );
}



2024-10-05 00:18:09 FINEST org.bsc.langgraph4j.CompiledGraph$AsyncNodeGenerator <init> START


INTERMEDIATE STEPS: []


2024-10-05 00:18:10 INFO REPL.$JShell$61 do_it$ STEP: NodeOutput{node=__START__, state={input=perform test once, intermediate_steps=[]}}
2024-10-05 00:18:10 INFO REPL.$JShell$61 do_it$ STEP: StateSnapshot{node=agent, state={input=perform test once, intermediate_steps=[], agent_outcome=AgentOutcome[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_TwegBFM5iEBqFKxXcl8WzKpl", name = "execTest", arguments = "{"message":"perform test once"}" }, log=], finish=null]}, config=RunnableConfig(threadId=test2, checkPointId=adb7d6ce-5d09-41cc-9984-27d0c73b6e6f, nextNode=action)}


INTERMEDIATE STEPS: [IntermediateStep[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_TwegBFM5iEBqFKxXcl8WzKpl", name = "execTest", arguments = "{"message":"perform test once"}" }, log=], observation=test tool executed: perform test once]]


2024-10-05 00:18:11 INFO REPL.$JShell$61 do_it$ STEP: StateSnapshot{node=action, state={input=perform test once, intermediate_steps=[IntermediateStep[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_TwegBFM5iEBqFKxXcl8WzKpl", name = "execTest", arguments = "{"message":"perform test once"}" }, log=], observation=test tool executed: perform test once]], agent_outcome=AgentOutcome[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_TwegBFM5iEBqFKxXcl8WzKpl", name = "execTest", arguments = "{"message":"perform test once"}" }, log=], finish=null]}, config=RunnableConfig(threadId=test2, checkPointId=d02625e3-44d1-4f2a-8e9a-9a3710956677, nextNode=agent)}
2024-10-05 00:18:11 INFO REPL.$JShell$61 do_it$ STEP: StateSnapshot{node=agent, state={input=perform test once, intermediate_steps=[IntermediateStep[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_TwegBFM5iEBqFKxXcl8WzKpl", name = "execTest", arguments = "{"message":"per

In [21]:
var history = graph.getStateHistory(config).stream().collect( Collectors.toList() );

var state2 =  history.get(2);

var updatedState = MapSerializer.of().cloneObject( state2.state().data() );

//System.out.println( state2.state().data() );
updatedState.put(  "input", "perform test twice"  );
//System.out.println( updatedState );

var updatedConfig = graph.updateState( state2.config(), updatedState );
log.info( "UPDATED CONFIG: {}", updatedConfig );

var iterator = graph.streamSnapshots( null, updatedConfig );  

try {
    for( var step : iterator ) {
        log.info( "STEP:\n {}", step );
    }    
}
catch( Exception e ) {
    Throwable t = e ;
    while( t.getCause() != null ) {
        t = t.getCause();
    }
    t.printStackTrace();
}


2024-10-05 00:18:11 INFO REPL.$JShell$63 do_it$ UPDATED CONFIG: RunnableConfig(threadId=test2, checkPointId=adb7d6ce-5d09-41cc-9984-27d0c73b6e6f, nextNode=null)
2024-10-05 00:18:11 FINEST org.bsc.langgraph4j.CompiledGraph$AsyncNodeGenerator <init> RESUME REQUEST
2024-10-05 00:18:11 FINEST org.bsc.langgraph4j.CompiledGraph$AsyncNodeGenerator <init> RESUME FROM agent


INTERMEDIATE STEPS: [IntermediateStep[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_TwegBFM5iEBqFKxXcl8WzKpl", name = "execTest", arguments = "{"message":"perform test once"}" }, log=], observation=test tool executed: perform test once]]


2024-10-05 00:18:12 INFO REPL.$JShell$64 do_it$ STEP:
 NodeOutput{node=action, state={input=perform test twice, intermediate_steps=[IntermediateStep[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_TwegBFM5iEBqFKxXcl8WzKpl", name = "execTest", arguments = "{"message":"perform test once"}" }, log=], observation=test tool executed: perform test once]], agent_outcome=AgentOutcome[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_TwegBFM5iEBqFKxXcl8WzKpl", name = "execTest", arguments = "{"message":"perform test once"}" }, log=], finish=null]}}
2024-10-05 00:18:12 INFO REPL.$JShell$64 do_it$ STEP:
 NodeOutput{node=agent, state={input=perform test twice, intermediate_steps=[IntermediateStep[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_TwegBFM5iEBqFKxXcl8WzKpl", name = "execTest", arguments = "{"message":"perform test once"}" }, log=], observation=test tool executed: perform test once]], agent_outcome=AgentOutcome[

INTERMEDIATE STEPS: [IntermediateStep[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_TwegBFM5iEBqFKxXcl8WzKpl", name = "execTest", arguments = "{"message":"perform test once"}" }, log=], observation=test tool executed: perform test once], IntermediateStep[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_S5LxTLBtpLdqVQcypdYdVCTF", name = "execTest", arguments = "{"message":"perform test again"}" }, log=], observation=test tool executed: perform test again]]


2024-10-05 00:18:13 INFO REPL.$JShell$64 do_it$ STEP:
 NodeOutput{node=action, state={input=perform test twice, intermediate_steps=[IntermediateStep[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_TwegBFM5iEBqFKxXcl8WzKpl", name = "execTest", arguments = "{"message":"perform test once"}" }, log=], observation=test tool executed: perform test once], IntermediateStep[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_S5LxTLBtpLdqVQcypdYdVCTF", name = "execTest", arguments = "{"message":"perform test again"}" }, log=], observation=test tool executed: perform test again]], agent_outcome=AgentOutcome[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_S5LxTLBtpLdqVQcypdYdVCTF", name = "execTest", arguments = "{"message":"perform test again"}" }, log=], finish=null]}}
2024-10-05 00:18:13 INFO REPL.$JShell$64 do_it$ STEP:
 NodeOutput{node=agent, state={input=perform test twice, intermediate_steps=[IntermediateStep[action=A